In [ ]:
# LIMITED READ
%matplotlib notebook
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [8, 8]

import pyodbc
import pandas as pd
import geopandas as gpd
import os
import pickle
from shapely.geometry import Point

In [ ]:
conn = pyodbc.connect('DSN=GEODWH;UID=GEODWH_READ;PWD=readonly')

ss_select = "SELECT [RPT_ID],[RIN],[RPT_TYPE],[SAMPLEID],[SAMPCODE],[FILE_ID],[LAT94],[LNG94],[As_ppm],[Au_ppm] FROM [GEODWH].[dbo].[ASSAY_SURFSAMP_TRACE] WHERE SAMPCODE in ('SOIL', 'AUGER', 'SURF_DRILL', 'LAG', 'FLOAT', 'UNKNOWN','VEGETATION')"
dh_select = "SELECT [RPT_ID],[RIN],[RPT_TYPE],[HOLEID] AS SAMPLEID,[DRILLCODE] AS SAMPCODE,[ASSAY_FILE_ID] AS FILEID,[LAT94],[LNG94],[As_ppm],[Au_ppm] FROM [GEODWH].[dbo].[ASSAY_MAX_DRILL_TRACE] WHERE DRILLCODE in ('AUGER', 'VAC', 'SURF_DRILL', 'VIB', 'AUG')"
confidence = "SELECT [MR_AUTHOR], [MR_CONFIDENTIALITY],[MR_RECTYPE],[MR_REPYEAR],[MR_RIN] FROM [GEODWH].[dbo].[DG_MINREPS]" #  where MR_RECTYPE = 'MINERAL'

dsfolder = os.path.join(os.path.dirname(os.getcwd()), 'data_src')

# NOTE: from ss: 'SOIL', 'AUGER', 'SURF_DRILL', 'LAG', 'FLOAT', 'UNKNOWN','VEGETATION'
# NOTE: from dh: 'AUGER', 'VAC', 'SURF_DRILL', 'VIB', 'AUG'


In [ ]:
# Read from the database
dh = pd.read_sql(dh_select, conn)
ss = pd.read_sql(ss_select, conn)
rpt_confidence = pd.read_sql(confidence, conn)

In [ ]:
# Fix the case insensitivity

dh['SAMPCODE'].replace('AUG', 'AUGER', inplace=True) # Fix a namecase
dh['SAMPCODE'].replace('Auger', 'AUGER', inplace=True) # Fix a namecase
dh['SAMPCODE'].replace('auger', 'AUGER', inplace=True) # Fix a namecase

dhtypes = dh['SAMPCODE'].unique()

print(dhtypes)

In [ ]:
# Fix the case insensitivity

ss['SAMPCODE'] = ss['SAMPCODE'].map(lambda x: x.strip())# Fix some whitespaces
ss['RIN'] = ss['RIN'].map(lambda x: x.strip())
ss['RPT_TYPE'] = ss['RPT_TYPE'].map(lambda x: x.strip())
ss['SAMPLEID'] = ss['SAMPLEID'].map(lambda x: x.strip())

ss['SAMPCODE'].replace('Soil', 'SOIL', inplace=True)
ss['SAMPCODE'].replace('soil', 'SOIL', inplace=True)

sstypes = ss['SAMPCODE'].unique()
print(sstypes)

In [ ]:
# Combine the drilling with the surface samples
samples = ss.append(dh)

# Check unique sample codes
stypes = samples['SAMPCODE'].unique()
stypes.sort()

print(f'Sample types: {stypes}')
print(len(samples))

In [ ]:
samples_m = pd.merge(samples, rpt_confidence, how='inner', left_on='RIN', right_on='MR_RIN', left_index=False, right_index=False)

samples_m.drop(['MR_RIN'], inplace=True, axis=1)
samples_m.drop(['MR_RECTYPE'], inplace=True, axis=1)

In [ ]:
len(samples_m)

In [ ]:
from pyproj import Proj
from pyproj import transform

geographic = Proj(init='epsg:4283')
projected = Proj(init='epsg:3112')

def lat(coords):
    E, N = transform(geographic, projected, coords[1], coords[0])
    return E

def lon(coords):
    E, N = transform(geographic, projected, coords[1], coords[0])
    return N

In [ ]:
samples['EASTING'] = samples.apply(lambda x: lat((x['LAT94'], x['LNG94'])), axis=1)
samples['NORTHING'] = samples.apply(lambda x: lon((x['LAT94'], x['LNG94'])), axis=1)

In [ ]:
# Make geometry field wkt (Well Known Text)
samples['Coordinates'] = list(zip(samples.EASTING, samples.NORTHING)) # Round to 6dp
samples['Coordinates'] = samples['Coordinates'].apply(Point)
samples = gpd.GeoDataFrame(samples, geometry='Coordinates', crs={'init' :'epsg:3112'})

samples.head()

In [ ]:
# Pickle them up for next time

with open(os.path.join(dsfolder, 'dh.pickle'), 'wb') as f:
    pickle.dump(dh, f, pickle.HIGHEST_PROTOCOL)
    
with open(os.path.join(dsfolder, 'ss.pickle'), 'wb') as f:
    pickle.dump(ss, f, pickle.HIGHEST_PROTOCOL)

with open(os.path.join(dsfolder, 'rpt_confidence.pickle'), 'wb') as f:
    pickle.dump(rpt_confidence, f, pickle.HIGHEST_PROTOCOL)

with open(os.path.join(dsfolder, 'samples_conditioned.pickle'), 'wb') as f:
    pickle.dump(samples, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# # Read the pickles

with open(os.path.join(dsfolder, 'samples_conditioned.pickle'), 'rb') as f:
    samps = pickle.load(f)
    
# with open(os.path.join(dsfolder, 'drillholes.pickle'), 'rb') as fin:
#     dh = pickle.load(fin)
    
# with open(os.path.join(dsfolder, 'ss.pickle'), 'rb') as fin:
#     ss = pickle.load(fin)
    
# with open(os.path.join(dsfolder, 'rpt_confidence.pickle'), 'rb') as fin:
#     rpt_confidence = pickle.load(fin)



In [ ]:
print(len(samps))

In [ ]:
samps.head()

In [ ]:
sampe[0:10000].plot()